<a href="https://colab.research.google.com/github/leon3108/Applied/blob/main/Applied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#upload CSV

# csv link : https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset
# you should download the link csv before run this cell
# when you have downloaded the csv you can run the cell, a window will pop up and will ask you for the file

from google.colab import files
uploaded = files.upload()

Saving diabetes_prediction_dataset.csv to diabetes_prediction_dataset.csv


In [ ]:
#import CSV on colab
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['diabetes_prediction_dataset.csv']))

In [ ]:
df.head()

# bmi(Body Mass Index) is a person's weight in kilograms (or pounds) divided by the square of height in meters (or feet)

# explain what is HbA1c_level is your average blood glucose (sugar) levels for the last two to three months

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
df["diabetes"].value_counts()
# data is imbalanced, adding weights to the class 1 could be an improvement but now I stay focus on train a basic model

0    91500
1     8500
Name: diabetes, dtype: int64

In [ ]:
print(df['smoking_history'].unique())
print(df['gender'].unique())

['never' 'No Info' 'current' 'former' 'ever' 'not current']
['Female' 'Male' 'Other']


In [ ]:
# as smoking_history and gender are categorical I need to modify the value to be numerical

df['gender'] = df['gender'].map({'Male': 0, 'Female': 1, "Other" : 2})
df['smoking_history'] = df['smoking_history'].map({'never': 0, 'No Info': 1, 'current': 2, 'former': 3, 'ever': 4, 'not current': 5})

In [ ]:
# choice of usefull feature
numerical_features = ["gender", "age", "hypertension", "heart_disease", "smoking_history", "bmi", "HbA1c_level", "blood_glucose_level"]
model_features = numerical_features

# choice of the target
model_target = 'diabetes'

In [ ]:
from sklearn.model_selection import train_test_split

# split the data for trainning and test/validation
X_train, X_val, y_train, y_val = train_test_split(df[model_features],
                                                  df[model_target],
                                                  test_size=0.10,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# pass the features in a mini-max
numerical_processor = Pipeline([
    ('num_scaler', MinMaxScaler())
])

data_preprocessor = ColumnTransformer([
    ('numerical_pre', numerical_processor, model_features),
])

# add logistic regression to be more precise
pipeline = Pipeline([
    ('data_preprocessing', data_preprocessor),
    ('logistic_regression', LogisticRegression(penalty = 'l2',
                              C = 0.1))
])

In [ ]:
# Visualizing the Pipeline
# Visual representation of the pipeline is enabled for clarity, especially useful for more complex pipelines.
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['gender', 'age',
                                                   'hypertension',
                                                   'heart_disease',
                                                   'smoking_history', 'bmi',
                                                   'HbA1c_level',
                                                   'blood_glucose_level'])])),
                ('logistic_regression', LogisticRegression(C=0.1))])

In [ ]:
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['gender', 'age',
                                                   'hypertension',
                                                   'heart_disease',
                                                   'smoking_history', 'bmi',
                                                   'HbA1c_level',
                                                   'blood_glucose_level'])])),
                ('logistic_regression', LogisticRegression(C=0.1))])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

val_predictions = pipeline.predict(X_val)

print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[9030   53]
 [ 369  548]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      9083
           1       0.91      0.60      0.72       917

    accuracy                           0.96     10000
   macro avg       0.94      0.80      0.85     10000
weighted avg       0.96      0.96      0.95     10000

Accuracy (validation): 0.9578
